# Similitud entre Productos - V2 (Con Fine-Tuning)

**Objetivo**: Determinar cuán similares son dos títulos de productos del dataset de test, generando un listado de pares ordenados por score de similitud.

**Modelo Base**: SBERT (paraphrase-multilingual-mpnet-base-v2)

**Fine-Tuning**: Entrenamiento con datos de `items_titles.csv` usando TSDAE (Transformer-based Sequential Denoising Auto-Encoder) para adaptación al dominio de productos.

**Criterio de filtrado**: Score de similitud >= 0.9

| Dataset | Productos | Uso |
|---------|-----------|-----|
| `items_titles.csv` | 30,000 | Fine-tuning del modelo |
| `items_titles_test.csv` | 10,000 | Evaluación y generación de pares |

---
## 1. Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
from datetime import datetime

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

from utils_similarity import (
    preprocess_dataframe,
    preprocess_title,
    analizar_longitud_titulos,
    ProductSimilarity,
    reduce_dimensions_3d,
    cluster_embeddings,
    plot_3d_clusters
)

# Configuración
THRESHOLD = 0.9  # Filtrar pares con similitud >= 0.9
FINETUNE_MODEL_PATH = './sbert_finetuned_productos'

print("Setup completado")

Setup completado


---
## 2. Cargar Datasets

In [2]:
# Cargar dataset de entrenamiento (para fine-tuning)
df_train = pd.read_csv('items_titles.csv')

# Cargar dataset de test
df_test = pd.read_csv('items_titles_test.csv')

print(f"Dataset TRAIN (fine-tuning): {len(df_train):,} productos")
print(f"Dataset TEST: {len(df_test):,} productos")
print(f"\nColumnas train: {df_train.columns.tolist()}")
print(f"Columnas test: {df_test.columns.tolist()}")

Dataset TRAIN (fine-tuning): 30,000 productos
Dataset TEST: 10,000 productos

Columnas train: ['ITE_ITEM_TITLE']
Columnas test: ['ITE_ITEM_TITLE']


In [3]:
# Muestra de títulos de entrenamiento
print("Ejemplos de títulos (train):")
for i, title in enumerate(df_train['ITE_ITEM_TITLE'].head(10)):
    print(f"  {i+1}. {title}")

Ejemplos de títulos (train):
  1. Tênis Ascension Posh Masculino - Preto E Vermelho 
  2. Tenis Para Caminhada Super Levinho Spider Corrida 
  3. Tênis Feminino Le Parc Hocks Black/ice Original Envio Já
  4. Tênis Olympikus Esportivo Academia Nova Tendência Triunfo 
  5. Inteligente Led Bicicleta Tauda Luz Usb Bicicleta Carregáve
  6. Tênis Casual Masculino Zarato 941 Preto 632
  7. Tênis Infantil Ortopasso Conforto Jogging
  8. Tv Samsung Qled 8k Q800t Semi Nova
  9. Tênis Usthemp Short Temático - Maria Vira-lata 2
  10. Sapatênis West Coast Urban Couro Masculino


In [4]:
# Estadísticas de longitud de títulos (train)
stats_train = analizar_longitud_titulos(df_train)
stats_test = analizar_longitud_titulos(df_test)

print("\nEstadísticas de Longitud (Train):")
print(f"  Caracteres: media={stats_train['char_length']['mean']:.1f}, mediana={stats_train['char_length']['median']:.0f}")
print(f"  Palabras:   media={stats_train['word_count']['mean']:.1f}, mediana={stats_train['word_count']['median']:.0f}")

print("\nEstadísticas de Longitud (Test):")
print(f"  Caracteres: media={stats_test['char_length']['mean']:.1f}, mediana={stats_test['char_length']['median']:.0f}")
print(f"  Palabras:   media={stats_test['word_count']['mean']:.1f}, mediana={stats_test['word_count']['median']:.0f}")


Estadísticas de Longitud (Train):
  Caracteres: media=47.0, mediana=50
  Palabras:   media=7.2, mediana=7

Estadísticas de Longitud (Test):
  Caracteres: media=47.0, mediana=50
  Palabras:   media=7.2, mediana=7


---
## 3. Preprocesamiento

In [5]:
# Preprocesar títulos de ambos datasets
df_train = preprocess_dataframe(df_train, column='ITE_ITEM_TITLE', output_column='title_clean')
df_test = preprocess_dataframe(df_test, column='ITE_ITEM_TITLE', output_column='title_clean')

# Mostrar ejemplos antes/después
print("Ejemplos de preprocesamiento (train):")
for i in range(5):
    print(f"\n  Original:    {df_train['ITE_ITEM_TITLE'].iloc[i]}")
    print(f"  Procesado:   {df_train['title_clean'].iloc[i]}")

Ejemplos de preprocesamiento (train):

  Original:    Tênis Ascension Posh Masculino - Preto E Vermelho 
  Procesado:   tênis ascension posh masculino preto e vermelho

  Original:    Tenis Para Caminhada Super Levinho Spider Corrida 
  Procesado:   tenis para caminhada super levinho spider corrida

  Original:    Tênis Feminino Le Parc Hocks Black/ice Original Envio Já
  Procesado:   tênis feminino le parc hocks black ice original envio já

  Original:    Tênis Olympikus Esportivo Academia Nova Tendência Triunfo 
  Procesado:   tênis olympikus esportivo academia nova tendência triunfo

  Original:    Inteligente Led Bicicleta Tauda Luz Usb Bicicleta Carregáve
  Procesado:   inteligente led bicicleta tauda luz usb bicicleta carregáve


In [6]:
# Preparar listas
titles_train_clean = df_train['title_clean'].tolist()
titles_train_original = df_train['ITE_ITEM_TITLE'].tolist()

titles_test_clean = df_test['title_clean'].tolist()
titles_test_original = df_test['ITE_ITEM_TITLE'].tolist()

print(f"\nTítulos para fine-tuning: {len(titles_train_clean):,}")
print(f"Títulos para evaluación: {len(titles_test_clean):,}")


Títulos para fine-tuning: 30,000
Títulos para evaluación: 10,000


---
## 4. Fine-Tuning del Modelo SBERT

Usamos **TSDAE (Transformer-based Sequential Denoising Auto-Encoder)** para fine-tuning no supervisado.

TSDAE funciona así:
1. Corrompe los textos de entrada (eliminando/permutando palabras)
2. El modelo aprende a reconstruir el texto original desde el embedding
3. Esto adapta el modelo al dominio específico de productos

In [7]:
# Verificar si ya existe un modelo fine-tuneado
if os.path.exists(FINETUNE_MODEL_PATH):
    print(f"✓ Modelo fine-tuneado encontrado en: {FINETUNE_MODEL_PATH}")
    SKIP_FINETUNING = True
else:
    print(f"✗ No se encontró modelo fine-tuneado. Se procederá con el entrenamiento.")
    SKIP_FINETUNING = False

✗ No se encontró modelo fine-tuneado. Se procederá con el entrenamiento.


In [8]:
%%time

if not SKIP_FINETUNING:
    from sentence_transformers import SentenceTransformer, InputExample, losses
    from sentence_transformers.datasets import DenoisingAutoEncoderDataset
    from torch.utils.data import DataLoader
    
    print("="*60)
    print("FINE-TUNING CON TSDAE")
    print("="*60)
    
    # Cargar modelo base
    print("\n1. Cargando modelo base...")
    model_base = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
    print(f"   Modelo: paraphrase-multilingual-mpnet-base-v2")
    print(f"   Dimensión: {model_base.get_sentence_embedding_dimension()}")
    
    # Crear dataset para TSDAE
    print("\n2. Preparando dataset para TSDAE...")
    train_sentences = titles_train_clean
    
    # Dataset con noise (denoising)
    train_dataset = DenoisingAutoEncoderDataset(train_sentences)
    
    # DataLoader
    train_dataloader = DataLoader(
        train_dataset, 
        batch_size=16, 
        shuffle=True,
        drop_last=True
    )
    print(f"   Samples: {len(train_sentences):,}")
    print(f"   Batches: {len(train_dataloader):,}")
    
    # Loss function para TSDAE
    print("\n3. Configurando TSDAE Loss...")
    train_loss = losses.DenoisingAutoEncoderLoss(
        model_base, 
        decoder_name_or_path='paraphrase-multilingual-mpnet-base-v2',
        tie_encoder_decoder=True
    )
    
    # Entrenar
    print("\n4. Iniciando entrenamiento...")
    print(f"   Epochs: 1")
    print(f"   Warmup steps: 100")
    
    model_base.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        warmup_steps=100,
        scheduler='warmupcosine',
        optimizer_params={'lr': 3e-5},
        show_progress_bar=True,
        use_amp=True  # Mixed precision para acelerar
    )
    
    # Guardar modelo
    print(f"\n5. Guardando modelo en: {FINETUNE_MODEL_PATH}")
    model_base.save(FINETUNE_MODEL_PATH)
    
    print("\n" + "="*60)
    print("✓ FINE-TUNING COMPLETADO")
    print("="*60)
else:
    print("Saltando fine-tuning (modelo ya existe)")

FINE-TUNING CON TSDAE

1. Cargando modelo base...


When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.


   Modelo: paraphrase-multilingual-mpnet-base-v2
   Dimensión: 768

2. Preparando dataset para TSDAE...
   Samples: 30,000
   Batches: 1,875

3. Configurando TSDAE Loss...


Some weights of XLMRobertaForCausalLM were not initialized from the model checkpoint at sentence-transformers/paraphrase-multilingual-mpnet-base-v2 and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.encoder.layer.0.crossattention.output.LayerNorm.bias', 'roberta.encoder.layer.0.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.0.crossattention.output.dense.bias', 'roberta.encoder.layer.0.crossattention.output.dense.weight', 'roberta.encoder.layer.0.crossattention.self.key.bias', 'roberta.encoder.layer.0.crossattention.self.key.weight', 'roberta.encoder.layer.0.crossattention.self.query.bias', 'roberta.encoder.layer.0.crossattention.self.query.weight', 'roberta.encoder.layer.0.crossattention.self.value.bias', 'roberta.encoder.layer.0.crossattention.self.value.weight', 'roberta.encoder.layer.1.crossattention.output.LayerNorm.bias', 'roberta.encoder


4. Iniciando entrenamiento...
   Epochs: 1
   Warmup steps: 100
CPU times: user 3.29 s, sys: 978 ms, total: 4.27 s
Wall time: 12.1 s


ImportError: Please install `datasets` to use this function: `pip install datasets`.

In [ ]:
# Importación de librerías y utilidades
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from utils_classifier import (
    # Constantes
    COST_FAILURE, COST_MAINTENANCE, ATTRIBUTES,
    # Clases principales
    FeatureEngineer, DataPreparator, ModelTrainer, OptunaOptimizer, Visualizer,
    # Funciones
    calculate_cost, calculate_baseline_cost, optimize_threshold,
    save_model_artifacts, predict_failure_probability,
    # Funciones de análisis
    analyze_dataset, analyze_nulls, analyze_target_distribution, plot_cost_over_time,
    plot_cost_over_time_with_model,
    analyze_temporal, analyze_baseline_costs, analyze_split, analyze_smote,
    analyze_threshold_optimization, analyze_final_model, print_executive_summary,
    analyze_mean_comparison, analyze_correlation, analyze_device_failures,
    select_best_boost_model, analyze_optimized_model_performance,
    analyze_attribute_distributions, analyze_feature_engineering_results, train_baseline_model,
    train_models_pipeline, run_optuna_optimization, run_threshold_optimization,
    evaluate_final_model_performance
)

pd.set_option('display.max_columns', None)
print("✅ Librerías importadas")

---
## 5. Generación de Embeddings con Modelo Fine-Tuneado

In [ ]:
%%time
from sentence_transformers import SentenceTransformer

# Cargar modelo fine-tuneado
print(f"Cargando modelo fine-tuneado desde: {FINETUNE_MODEL_PATH}")
model_finetuned = SentenceTransformer(FINETUNE_MODEL_PATH)
print(f"Modelo cargado. Dimensión: {model_finetuned.get_sentence_embedding_dimension()}")

# Generar embeddings para el dataset de test
print(f"\nGenerando embeddings para {len(titles_test_clean):,} productos...")
embeddings = model_finetuned.encode(
    titles_test_clean,
    show_progress_bar=True,
    batch_size=64
)

print(f"\nEmbeddings shape: {embeddings.shape}")

---
## 6. Visualización 3D de Clusters

In [ ]:
# Reducir dimensiones para visualización
print("Reduciendo dimensiones a 3D...")
embeddings_3d = reduce_dimensions_3d(embeddings, method='pca')

In [ ]:
# Clustering
N_CLUSTERS = 5
clusters = cluster_embeddings(embeddings, n_clusters=N_CLUSTERS)

# Distribución de clusters
print("\nDistribución de productos por cluster:")
for i in range(N_CLUSTERS):
    count = (clusters == i).sum()
    print(f"  Cluster {i}: {count:,} productos ({count/len(clusters)*100:.1f}%)")

In [ ]:
# Visualización 3D
fig = plot_3d_clusters(
    embeddings_3d=embeddings_3d,
    clusters=clusters,
    titles=titles_test_clean,
    title_plot='Clusters de Productos (Modelo Fine-Tuneado)'
)
fig.show()

---
## 7. Cálculo de Similitud y Filtrado

In [ ]:
%%time
from sklearn.metrics.pairwise import cosine_similarity

# Calcular matriz de similitud completa
print(f"Calculando matriz de similitud ({len(embeddings):,} x {len(embeddings):,})...")
sim_matrix = cosine_similarity(embeddings)
print(f"Matriz calculada. Shape: {sim_matrix.shape}")

In [ ]:
# Extraer pares del triángulo superior (evitar duplicados)
n = len(titles_test_original)
upper_tri_indices = np.triu_indices(n, k=1)
scores = sim_matrix[upper_tri_indices]

# Filtrar por threshold
mask = scores >= THRESHOLD
filtered_indices = np.where(mask)[0]

print(f"Total de pares posibles: {len(scores):,}")
print(f"Pares con score >= {THRESHOLD}: {len(filtered_indices):,}")

In [ ]:
# Crear DataFrame con los pares filtrados
results = []
for idx in filtered_indices:
    i = upper_tri_indices[0][idx]
    j = upper_tri_indices[1][idx]
    results.append({
        'ITE_ITEM_TITLE': titles_test_original[i],
        'ITE_ITEM_TITLE_2': titles_test_original[j],
        'Score Similitud (0,1)': round(scores[idx], 4)
    })

df_output = pd.DataFrame(results)

# Ordenar por score descendente
df_output = df_output.sort_values('Score Similitud (0,1)', ascending=False).reset_index(drop=True)

print(f"\nPares similares encontrados: {len(df_output):,}")

In [ ]:
# Vista previa de los pares más similares
print("\nTop 10 pares más similares:")
for i, row in df_output.head(10).iterrows():
    print(f"\n[{row['Score Similitud (0,1)']:.4f}]")
    print(f"  1: {row['ITE_ITEM_TITLE'][:70]}...")
    print(f"  2: {row['ITE_ITEM_TITLE_2'][:70]}...")

In [ ]:
# Distribución de scores
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(df_output['Score Similitud (0,1)'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax.axvline(df_output['Score Similitud (0,1)'].mean(), color='red', linestyle='--', 
          label=f"Media: {df_output['Score Similitud (0,1)'].mean():.4f}")
ax.set_xlabel('Score de Similitud')
ax.set_ylabel('Frecuencia')
ax.set_title(f'Distribución de Scores (threshold >= {THRESHOLD}) - Modelo Fine-Tuneado')
ax.legend()
plt.tight_layout()
plt.show()

print(f"\nEstadísticas de scores filtrados:")
print(df_output['Score Similitud (0,1)'].describe())

---
## 8. Comparación: Modelo Base vs Fine-Tuneado

In [ ]:
%%time
# Cargar modelo base (sin fine-tuning) para comparación
print("Cargando modelo base para comparación...")
model_base = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

# Generar embeddings con modelo base
print(f"Generando embeddings con modelo BASE...")
embeddings_base = model_base.encode(
    titles_test_clean,
    show_progress_bar=True,
    batch_size=64
)
print(f"Embeddings BASE shape: {embeddings_base.shape}")

In [ ]:
# Calcular similitudes con modelo base
sim_matrix_base = cosine_similarity(embeddings_base)
scores_base = sim_matrix_base[upper_tri_indices]

# Filtrar por threshold
mask_base = scores_base >= THRESHOLD
filtered_indices_base = np.where(mask_base)[0]

print(f"\nComparación de resultados:")
print(f"  Modelo BASE: {len(filtered_indices_base):,} pares con score >= {THRESHOLD}")
print(f"  Modelo FINE-TUNED: {len(filtered_indices):,} pares con score >= {THRESHOLD}")

In [ ]:
# Estadísticas comparativas
print("\nEstadísticas de Similitud:")
print("\nModelo BASE:")
print(f"  Media: {scores_base.mean():.4f}")
print(f"  Std:   {scores_base.std():.4f}")
print(f"  Min:   {scores_base.min():.4f}")
print(f"  Max:   {scores_base.max():.4f}")

print("\nModelo FINE-TUNED:")
print(f"  Media: {scores.mean():.4f}")
print(f"  Std:   {scores.std():.4f}")
print(f"  Min:   {scores.min():.4f}")
print(f"  Max:   {scores.max():.4f}")

In [ ]:
# Visualización comparativa
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma modelo base
axes[0].hist(scores_base, bins=100, alpha=0.7, color='steelblue', edgecolor='black')
axes[0].axvline(scores_base.mean(), color='red', linestyle='--', label=f'Media: {scores_base.mean():.4f}')
axes[0].axvline(THRESHOLD, color='green', linestyle=':', label=f'Threshold: {THRESHOLD}')
axes[0].set_xlabel('Score de Similitud')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Modelo BASE')
axes[0].legend()

# Histograma modelo fine-tuned
axes[1].hist(scores, bins=100, alpha=0.7, color='coral', edgecolor='black')
axes[1].axvline(scores.mean(), color='red', linestyle='--', label=f'Media: {scores.mean():.4f}')
axes[1].axvline(THRESHOLD, color='green', linestyle=':', label=f'Threshold: {THRESHOLD}')
axes[1].set_xlabel('Score de Similitud')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title('Modelo FINE-TUNED')
axes[1].legend()

plt.suptitle('Comparación: Distribución de Similitudes', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot: correlación entre scores de ambos modelos
# Tomar muestra para visualización
sample_size = 10000
np.random.seed(42)
sample_idx = np.random.choice(len(scores), min(sample_size, len(scores)), replace=False)

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(scores_base[sample_idx], scores[sample_idx], alpha=0.3, s=10)
ax.plot([0, 1], [0, 1], 'r--', label='Referencia (x=y)')
ax.axhline(THRESHOLD, color='green', linestyle=':', alpha=0.5, label=f'Threshold: {THRESHOLD}')
ax.axvline(THRESHOLD, color='green', linestyle=':', alpha=0.5)
ax.set_xlabel('Score Modelo BASE')
ax.set_ylabel('Score Modelo FINE-TUNED')
ax.set_title(f'Correlación de Scores\n(muestra de {len(sample_idx):,} pares)')
ax.legend()

# Calcular correlación
correlation = np.corrcoef(scores_base[sample_idx], scores[sample_idx])[0, 1]
ax.text(0.05, 0.95, f'Correlación: {correlation:.4f}', transform=ax.transAxes, 
        fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

---
## 9. Exportar CSV

In [ ]:
# Preparar output final
df_final = df_output.copy()
df_final.columns = ['ITE_ITEM_TITLE', 'ITE_ITEM_TITLE_2', 'Score Similitud (0,1)']

# Guardar
OUTPUT_FILE = 'output_similitud_finetuned.csv'
df_output.to_csv(OUTPUT_FILE, index=False)

print(f"Archivo guardado: {OUTPUT_FILE}")
print(f"Total de pares: {len(df_output):,}")

In [ ]:
# También guardar resultados del modelo base para comparación
results_base = []
for idx in filtered_indices_base:
    i = upper_tri_indices[0][idx]
    j = upper_tri_indices[1][idx]
    results_base.append({
        'ITE_ITEM_TITLE': titles_test_original[i],
        'ITE_ITEM_TITLE_2': titles_test_original[j],
        'Score Similitud (0,1)': round(scores_base[idx], 4)
    })

df_output_base = pd.DataFrame(results_base)
df_output_base = df_output_base.sort_values('Score Similitud (0,1)', ascending=False).reset_index(drop=True)

OUTPUT_FILE_BASE = 'output_similitud_base.csv'
df_output_base.to_csv(OUTPUT_FILE_BASE, index=False)

print(f"\nArchivo modelo base guardado: {OUTPUT_FILE_BASE}")
print(f"Total de pares (base): {len(df_output_base):,}")

---
## 10. Resumen

In [ ]:
print("="*60)
print("RESUMEN - SIMILITUD ENTRE PRODUCTOS V2 (CON FINE-TUNING)")
print("="*60)

print(f"\nDatasets:")
print(f"  - Train (fine-tuning): {len(df_train):,} productos")
print(f"  - Test (evaluación): {len(df_test):,} productos")

print(f"\nModelo:")
print(f"  - Base: paraphrase-multilingual-mpnet-base-v2")
print(f"  - Fine-tuning: TSDAE (Denoising Auto-Encoder)")
print(f"  - Dimensión embeddings: {embeddings.shape[1]}")

print(f"\nResultados Modelo FINE-TUNED (threshold >= {THRESHOLD}):")
print(f"  - Pares encontrados: {len(df_output):,}")
print(f"  - Score máximo: {df_output['Score Similitud (0,1)'].max():.4f}")
print(f"  - Score mínimo: {df_output['Score Similitud (0,1)'].min():.4f}")
print(f"  - Score medio: {df_output['Score Similitud (0,1)'].mean():.4f}")

print(f"\nResultados Modelo BASE (threshold >= {THRESHOLD}):")
print(f"  - Pares encontrados: {len(df_output_base):,}")
print(f"  - Score máximo: {df_output_base['Score Similitud (0,1)'].max():.4f}")
print(f"  - Score mínimo: {df_output_base['Score Similitud (0,1)'].min():.4f}")
print(f"  - Score medio: {df_output_base['Score Similitud (0,1)'].mean():.4f}")

print(f"\nArchivos de salida:")
print(f"  - Fine-tuned: {OUTPUT_FILE}")
print(f"  - Base: {OUTPUT_FILE_BASE}")
print(f"  - Modelo: {FINETUNE_MODEL_PATH}")
print("="*60)